<a href="https://colab.research.google.com/github/SilverFoxMedia-Tesis/IA_01/blob/main/Algoritmo_Regresion_Lineal_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/ObesityDataSet_raw_and_data_sinthetic.csv')

train = df[:2000]
test = df[2000:]

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [ ]:
train = pd.read_csv('/train.csv')
test = pd.read_csv('/test.csv')

VARIABLES POR INDICE DE MASA CORPORAL

IMC = ALTURA / PESO^2

Debajo del peso ideal: Menos de 18.5

Normal: 18.5 - 24.9

Sobrepeso: 25.0 - 29.9

Obesidad 1: 30.0 - 34.9

Obesidad 2: 35.0 - 39.9

Obesidad 3: Más de 40

In [ ]:
estadoxvalores = {
    "Insufficient_Weight": 0,
    "Normal_Weight": 0,
    "Overweight_Level_I": -1,
    "Overweight_Level_II": -2,
    "Obesity_Type_I": -3,
    "Obesity_Type_II": -4,
    "Obesity_Type_III": -5,
}
FAFxvalores = {
    3 : 1,
    2 : 0,
    1 : -1,
    0 : -2,
}
TUExvalores = {
    0 : 0,
    1 : -1,
    2 : -2,
}
MTRANSxvalores = {
    "Bike": 2,
    "Walking": 1,
    "Public_Transportation": 0,
    "Motorbike": -1,
    "Automobile": -2,
}
def encontrar_nivel(puntos_nivel):
    if -11 <= puntos_nivel <= -7:
        return "Beginner"
    elif -6 <= puntos_nivel <= -2:
        return "Intermediate"
    elif -1 <= puntos_nivel <= 3:
        return "Expert"
    else:
        return "Desconocido"

In [ ]:
train['Puntos_Nivel'] = train['MTRANS'].map(MTRANSxvalores) + train['NObeyesdad'].map(estadoxvalores) + train['TUE'].apply(lambda x: TUExvalores.get(int(x), 0)) + train['FAF'].apply(lambda x: FAFxvalores.get(int(x), 2))
train['Nivel'] = train['Puntos_Nivel'].apply(encontrar_nivel)

In [ ]:
test['Puntos_Nivel'] = test['MTRANS'].map(MTRANSxvalores) + test['NObeyesdad'].map(estadoxvalores) + test['TUE'].apply(lambda x: TUExvalores.get(int(x), 0)) + test['FAF'].apply(lambda x: FAFxvalores.get(int(x), 2))

In [ ]:
train

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,Puntos_Nivel,Nivel
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight,-3,Intermediate
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight,1,Expert
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight,-1,Expert
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I,0,Expert
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II,-4,Intermediate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Female,20.388049,1.777971,137.785027,yes,yes,3.0,3.0,Sometimes,no,2.702789,no,1.606109,0.731213,Sometimes,Public_Transportation,Obesity_Type_III,-6,Intermediate
1996,Female,21.022206,1.739950,135.693381,yes,yes,3.0,3.0,Sometimes,no,1.663213,no,1.094035,0.786665,Sometimes,Public_Transportation,Obesity_Type_III,-6,Intermediate
1997,Female,20.102241,1.816868,153.959945,yes,yes,3.0,3.0,Sometimes,no,2.414739,no,1.917014,0.927982,Sometimes,Public_Transportation,Obesity_Type_III,-6,Intermediate
1998,Female,21.291969,1.800200,155.242672,yes,yes,3.0,3.0,Sometimes,no,2.351193,no,1.051889,0.686491,Sometimes,Public_Transportation,Obesity_Type_III,-6,Intermediate


-11 <= BEGINNER <= -7

-6 <= INTERMEDIATE <= -2

-1 <= EXPERT <= 3

**Atributos relacionados con el peso:**

- *Calories consumption monitoring (SCC)*
-- YES
-- NO


- *Physical activity frequency (FAF)*
-- NUMERIC

- *Time using technology devices (TUE)*
-- NUMERIC

- *Transportation used (MTRANS)*
-- AUTOMOBILE
-- MOTORBIKE
-- BIKE
-- PUBLIC_TRANSPORTATION
-- WALKING

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Preparación de los conjuntos de datos
X_train = train.drop('Nivel', axis=1)  # Características del conjunto de datos con columna extra
y_train = train['Nivel']  # Valores objetivo (columna extra)

X_test = test  # Características del conjunto de datos sin columna extra

# Identificar columnas categóricas
columnas_categoricas = X_train.select_dtypes(include=['object']).columns.tolist()

# Aplicar codificación one-hot a las columnas categóricas
transformer = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), columnas_categoricas)], remainder='passthrough')
X_train_encoded = transformer.fit_transform(X_train)
X_test_encoded = transformer.transform(X_test)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Crear una instancia del codificador ordinal
encoder = OrdinalEncoder()

# Ajustar y transformar la variable objetivo
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Crear una instancia del modelo de regresión lineal
modelo = LinearRegression()

# Entrenar el modelo utilizando los datos de entrenamiento codificados
modelo.fit(X_train_encoded, y_train_encoded)

# Realizar predicciones en el conjunto de datos de prueba codificado
y_pred = modelo.predict(X_test_encoded)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# Seleccionar la última columna de y_test
y_test_last_column = X_test_encoded[:, -1]

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test_last_column, y_pred)

# Calcular el coeficiente de determinación
r2 = r2_score(y_test_last_column, y_pred)

# Imprimir las métricas de evaluación
print("Error cuadrático medio:", mse)
print("Coeficiente de determinación:", r2)

Error cuadrático medio: 53.626296063283874
Coeficiente de determinación: -217.06257220980874
